In [1]:
import requests
import os
import json
import pandas as pd
import expansions

In [2]:
bearer_token = os.environ.get("BEARER_TOKEN")

In [3]:
def create_url(): 
    user_id = 2244994945
    return "https://api.twitter.com/2/users/{}/tweets".format(user_id)

In [4]:
url = create_url()
print(url)

https://api.twitter.com/2/users/2244994945/tweets


In [5]:
params = {"tweet.fields": "attachments,author_id,context_annotations,conversation_id,created_at,entities,geo,id,in_reply_to_user_id,lang,possibly_sensitive,public_metrics,referenced_tweets,reply_settings,source,text,withheld", "user.fields": "created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld", "expansions": "author_id,referenced_tweets.id,referenced_tweets.id.author_id,entities.mentions.username,attachments.poll_ids,attachments.media_keys,in_reply_to_user_id,geo.place_id", "place.fields": "contained_within,country,country_code,full_name,geo,id,name,place_type", "poll.fields": "duration_minutes,end_datetime,id,options,voting_status", "max_results": "5"}

In [6]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [7]:
headers = create_headers(bearer_token)

In [8]:
def connect_to_endpoint(url, headers, params):
    response = requests.request("GET", url, headers=headers, params=params)
    print(response.status_code)
    return response.json()

In [9]:
response = connect_to_endpoint(url, headers, params)

401


In [10]:
print(response)

{'title': 'Unauthorized', 'type': 'about:blank', 'status': 401, 'detail': 'Unauthorized'}


In [11]:
json_response = expansions.flatten(response)
df = pd.json_normalize(json_response['data'], max_level=4)
df

KeyError: 'data'

In [12]:
new_df = df['referenced_tweets'].apply(pd.Series)
new_df.columns = ['rt']
new_df

NameError: name 'df' is not defined

In [34]:
norm = new_df['rt'].apply(pd.Series)
norm

,0,author,author_id,context_annotations,conversation_id,created_at,entities,id,in_reply_to_user,in_reply_to_user_id,lang,possibly_sensitive,public_metrics,referenced_tweets,reply_settings,source,text,type
0,NaN,"{'username': 'TwitterDev', 'id': '2244994945',...",2244994945,"[{'domain': {'id': '46', 'name': 'Brand Catego...",1370110639545417729,2021-03-11T20:33:26.000Z,"{'hashtags': [{'start': 110, 'end': 121, 'tag'...",1370110639545417729,NaN,NaN,en,False,"{'retweet_count': 11, 'reply_count': 0, 'like_...",NaN,everyone,Twitter Web App,"On Tuesday at 12 pm EST, @jessicagarson will c...",quoted
1,NaN,"{'username': 'TwitterDev', 'id': '2244994945',...",2244994945,"[{'domain': {'id': '46', 'name': 'Brand Catego...",1370110639545417729,2021-03-11T20:33:26.000Z,"{'hashtags': [{'start': 110, 'end': 121, 'tag'...",1370110639545417729,NaN,NaN,en,False,"{'retweet_count': 11, 'reply_count': 0, 'like_...",NaN,everyone,Twitter Web App,"On Tuesday at 12 pm EST, @jessicagarson will c...",quoted
2,NaN,"{'username': 'suhemparack', 'id': '85769996926...",857699969263964161,"[{'domain': {'id': '46', 'name': 'Brand Catego...",1367897834758934530,2021-03-05T18:00:32.000Z,"{'hashtags': [{'start': 58, 'end': 69, 'tag': ...",1367897834758934530,NaN,NaN,en,False,"{'retweet_count': 19, 'reply_count': 2, 'like_...",NaN,everyone,Twitter Web App,🚨 Approved for the Academic Research product t...,quoted
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,"{'username': 'TwitterDev', 'id': '2244994945',...",2244994945,"[{'domain': {'id': '46', 'name': 'Brand Catego...",1370028371585814532,2021-03-11T15:06:32.000Z,"{'annotations': [{'start': 13, 'end': 20, 'pro...",1370028372701585408,"{'username': 'TwitterDev', 'id': '2244994945',...",2244994945,en,False,"{'retweet_count': 0, 'reply_count': 2, 'like_c...","[{'type': 'replied_to', 'id': '137002837158581...",everyone,Twitter Web App,“The idea of Threader came from the desire to ...,replied_to


In [36]:
df['referenced_tweets.id'] = norm['id']
df['referenced_tweets.type'] = norm['type'] 
df

,id,referenced_tweets,lang,author_id,reply_settings,possibly_sensitive,created_at,text,source,context_annotations,...,in_reply_to_user.verified,in_reply_to_user.created_at,in_reply_to_user.public_metrics.followers_count,in_reply_to_user.public_metrics.following_count,in_reply_to_user.public_metrics.tweet_count,in_reply_to_user.public_metrics.listed_count,in_reply_to_user.entities.url.urls,in_reply_to_user.entities.description.hashtags,referenced_tweets.id,referenced_tweets.type
0,1371853548762333187,"[{'type': 'quoted', 'id': '1370110639545417729...",en,2244994945,everyone,False,2021-03-16T15:59:08.000Z,Live now! https://t.co/sC2M3eRpPk,Twitter Web App,"[{'domain': {'id': '46', 'name': 'Brand Catego...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1370110639545417729,quoted
1,1371588329368657923,"[{'type': 'quoted', 'id': '1370110639545417729...",en,2244994945,everyone,False,2021-03-15T22:25:15.000Z,Hope to see you tomorrow at 12 pm EST! \nhttps...,Twitter Web App,"[{'domain': {'id': '46', 'name': 'Brand Catego...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1370110639545417729,quoted
2,1371363033352638467,"[{'type': 'quoted', 'id': '1367897834758934530...",en,2244994945,everyone,False,2021-03-15T07:30:00.000Z,🔎 If you're using v2 full-archive search for A...,Twitter Web App,"[{'domain': {'id': '46', 'name': 'Brand Catego...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1367897834758934530,quoted
3,1370110639545417729,NaN,en,2244994945,everyone,False,2021-03-11T20:33:26.000Z,"On Tuesday at 12 pm EST, @jessicagarson will c...",Twitter Web App,"[{'domain': {'id': '46', 'name': 'Brand Catego...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1370028373792071687,"[{'type': 'replied_to', 'id': '137002837270158...",en,2244994945,everyone,False,2021-03-11T15:06:32.000Z,"“I have been a developer from a young age,” sa...",Twitter Web App,"[{'domain': {'id': '46', 'name': 'Brand Catego...",...,True,2013-12-14T04:35:55.000Z,517316.0,2034.0,3680.0,1725.0,"[{'start': 0, 'end': 23, 'url': 'https://t.co/...","[{'start': 17, 'end': 28, 'tag': 'TwitterDev'}...",1370028372701585408,replied_to


In [39]:
df = df.drop(columns='referenced_tweets')

KeyError: "['referenced_tweets'] not found in axis"

In [40]:
df

,id,lang,author_id,reply_settings,possibly_sensitive,created_at,text,source,context_annotations,conversation_id,...,in_reply_to_user.verified,in_reply_to_user.created_at,in_reply_to_user.public_metrics.followers_count,in_reply_to_user.public_metrics.following_count,in_reply_to_user.public_metrics.tweet_count,in_reply_to_user.public_metrics.listed_count,in_reply_to_user.entities.url.urls,in_reply_to_user.entities.description.hashtags,referenced_tweets.id,referenced_tweets.type
0,1371853548762333187,en,2244994945,everyone,False,2021-03-16T15:59:08.000Z,Live now! https://t.co/sC2M3eRpPk,Twitter Web App,"[{'domain': {'id': '46', 'name': 'Brand Catego...",1371853548762333187,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1370110639545417729,quoted
1,1371588329368657923,en,2244994945,everyone,False,2021-03-15T22:25:15.000Z,Hope to see you tomorrow at 12 pm EST! \nhttps...,Twitter Web App,"[{'domain': {'id': '46', 'name': 'Brand Catego...",1371588329368657923,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1370110639545417729,quoted
2,1371363033352638467,en,2244994945,everyone,False,2021-03-15T07:30:00.000Z,🔎 If you're using v2 full-archive search for A...,Twitter Web App,"[{'domain': {'id': '46', 'name': 'Brand Catego...",1371363033352638467,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1367897834758934530,quoted
3,1370110639545417729,en,2244994945,everyone,False,2021-03-11T20:33:26.000Z,"On Tuesday at 12 pm EST, @jessicagarson will c...",Twitter Web App,"[{'domain': {'id': '46', 'name': 'Brand Catego...",1370110639545417729,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1370028373792071687,en,2244994945,everyone,False,2021-03-11T15:06:32.000Z,"“I have been a developer from a young age,” sa...",Twitter Web App,"[{'domain': {'id': '46', 'name': 'Brand Catego...",1370028371585814532,...,True,2013-12-14T04:35:55.000Z,517316.0,2034.0,3680.0,1725.0,"[{'start': 0, 'end': 23, 'url': 'https://t.co/...","[{'start': 17, 'end': 28, 'tag': 'TwitterDev'}...",1370028372701585408,replied_to


In [41]:
context_annotations = pd.concat([pd.DataFrame(pd.json_normalize(x)) for x in df['context_annotations']],ignore_index=True)
context_annotations

,domain.id,domain.name,domain.description,entity.id,entity.name,entity.description
0,46,Brand Category,Categories within Brand Verticals that narrow ...,781974596752842752,Services,NaN
1,47,Brand,Brands and Companies,10045225402,Twitter,NaN
2,47,Brand,Brands and Companies,1054826545758396416,Twitch,Twitch
3,65,Interests and Hobbies Vertical,"Top level interests and hobbies groupings, lik...",848920371311001600,Technology,Technology and computing
4,66,Interests and Hobbies Category,"A grouping of interests and hobbies entities, ...",848921413196984320,Computer programming,Computer programming
5,46,Brand Category,Categories within Brand Verticals that narrow ...,781974596752842752,Services,NaN
6,47,Brand,Brands and Companies,10045225402,Twitter,NaN
7,65,Interests and Hobbies Vertical,"Top level interests and hobbies groupings, lik...",848920371311001600,Technology,Technology and computing
8,66,Interests and Hobbies Category,"A grouping of interests and hobbies entities, ...",848921413196984320,Computer programming,Computer programming
9,46,Brand Category,Categories within Brand Verticals that narrow ...,781974596752842752,Services,NaN


In [43]:
df = df.merge(context_annotations, left_index=True, right_index=True)
df

,id,lang,author_id,reply_settings,possibly_sensitive,created_at,text,source,context_annotations,conversation_id,...,domain.description_x,entity.id_x,entity.name_x,entity.description_x,domain.id_y,domain.name_y,domain.description_y,entity.id_y,entity.name_y,entity.description_y
0,1371853548762333187,en,2244994945,everyone,False,2021-03-16T15:59:08.000Z,Live now! https://t.co/sC2M3eRpPk,Twitter Web App,"[{'domain': {'id': '46', 'name': 'Brand Catego...",1371853548762333187,...,Categories within Brand Verticals that narrow ...,781974596752842752,Services,NaN,46,Brand Category,Categories within Brand Verticals that narrow ...,781974596752842752,Services,NaN
1,1371588329368657923,en,2244994945,everyone,False,2021-03-15T22:25:15.000Z,Hope to see you tomorrow at 12 pm EST! \nhttps...,Twitter Web App,"[{'domain': {'id': '46', 'name': 'Brand Catego...",1371588329368657923,...,Brands and Companies,10045225402,Twitter,NaN,47,Brand,Brands and Companies,10045225402,Twitter,NaN
2,1371363033352638467,en,2244994945,everyone,False,2021-03-15T07:30:00.000Z,🔎 If you're using v2 full-archive search for A...,Twitter Web App,"[{'domain': {'id': '46', 'name': 'Brand Catego...",1371363033352638467,...,Brands and Companies,1054826545758396416,Twitch,Twitch,47,Brand,Brands and Companies,1054826545758396416,Twitch,Twitch
3,1370110639545417729,en,2244994945,everyone,False,2021-03-11T20:33:26.000Z,"On Tuesday at 12 pm EST, @jessicagarson will c...",Twitter Web App,"[{'domain': {'id': '46', 'name': 'Brand Catego...",1370110639545417729,...,"Top level interests and hobbies groupings, lik...",848920371311001600,Technology,Technology and computing,65,Interests and Hobbies Vertical,"Top level interests and hobbies groupings, lik...",848920371311001600,Technology,Technology and computing
4,1370028373792071687,en,2244994945,everyone,False,2021-03-11T15:06:32.000Z,"“I have been a developer from a young age,” sa...",Twitter Web App,"[{'domain': {'id': '46', 'name': 'Brand Catego...",1370028371585814532,...,"A grouping of interests and hobbies entities, ...",848921413196984320,Computer programming,Computer programming,66,Interests and Hobbies Category,"A grouping of interests and hobbies entities, ...",848921413196984320,Computer programming,Computer programming


In [44]:
df = df.drop(columns='context_annotations')
df

,id,lang,author_id,reply_settings,possibly_sensitive,created_at,text,source,conversation_id,public_metrics.retweet_count,...,domain.description_x,entity.id_x,entity.name_x,entity.description_x,domain.id_y,domain.name_y,domain.description_y,entity.id_y,entity.name_y,entity.description_y
0,1371853548762333187,en,2244994945,everyone,False,2021-03-16T15:59:08.000Z,Live now! https://t.co/sC2M3eRpPk,Twitter Web App,1371853548762333187,2,...,Categories within Brand Verticals that narrow ...,781974596752842752,Services,NaN,46,Brand Category,Categories within Brand Verticals that narrow ...,781974596752842752,Services,NaN
1,1371588329368657923,en,2244994945,everyone,False,2021-03-15T22:25:15.000Z,Hope to see you tomorrow at 12 pm EST! \nhttps...,Twitter Web App,1371588329368657923,0,...,Brands and Companies,10045225402,Twitter,NaN,47,Brand,Brands and Companies,10045225402,Twitter,NaN
2,1371363033352638467,en,2244994945,everyone,False,2021-03-15T07:30:00.000Z,🔎 If you're using v2 full-archive search for A...,Twitter Web App,1371363033352638467,5,...,Brands and Companies,1054826545758396416,Twitch,Twitch,47,Brand,Brands and Companies,1054826545758396416,Twitch,Twitch
3,1370110639545417729,en,2244994945,everyone,False,2021-03-11T20:33:26.000Z,"On Tuesday at 12 pm EST, @jessicagarson will c...",Twitter Web App,1370110639545417729,11,...,"Top level interests and hobbies groupings, lik...",848920371311001600,Technology,Technology and computing,65,Interests and Hobbies Vertical,"Top level interests and hobbies groupings, lik...",848920371311001600,Technology,Technology and computing
4,1370028373792071687,en,2244994945,everyone,False,2021-03-11T15:06:32.000Z,"“I have been a developer from a young age,” sa...",Twitter Web App,1370028371585814532,0,...,"A grouping of interests and hobbies entities, ...",848921413196984320,Computer programming,Computer programming,66,Interests and Hobbies Category,"A grouping of interests and hobbies entities, ...",848921413196984320,Computer programming,Computer programming


In [45]:
df.to_csv('final_df.csv')